# Programming Assignment: Recommender Systems

## Imports

In [1044]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random
from collections import defaultdict

## Part 1: Data Processing

We first read the raw data in using the correct delimiter and give column names for the data. The data is then saved in csv format.

In [1045]:
udata = pd.read_csv('./u.data', delimiter='\t', names=["user_id", "movie_id", "rating", "timestamp"])

uitem = pd.read_csv('./u.item', delimiter='|', encoding='latin1', usecols=[0, 1, 2], header=None)
uitem.columns = ["movie_id", "title", "release_date"]
uitem["movie_id"] = uitem["movie_id"].astype(int)

uuser = pd.read_csv('./u.user', delimiter='|', encoding='latin1', usecols=[0, 1, 2, 3], header=None)
uuser.columns = ["user_id", "age", "gender", "occupation"]

udata.to_csv('udata.csv', index=False)
uitem.to_csv('uitem.csv', index=False)
uuser.to_csv('uuser.csv', index=False)

In [1046]:
print(udata.head(3))
print()
print(uitem.head(3))
print()
print(uuser.head(3))

   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116

   movie_id              title release_date
0         1   Toy Story (1995)  01-Jan-1995
1         2   GoldenEye (1995)  01-Jan-1995
2         3  Four Rooms (1995)  01-Jan-1995

   user_id  age gender  occupation
0        1   24      M  technician
1        2   53      F       other
2        3   23      M      writer


Load the `udata` csv file from the files that were just created and create a matrix, `user_movie_matrix`, with user ratings for each movie using the data in `udata`. User IDs are the rows and movie IDs are the columns.

In [1047]:
udata = pd.read_csv('udata.csv')

user_movie_matrix = udata.pivot(index="user_id", columns="movie_id", values="rating")
user_movie_matrix.columns = user_movie_matrix.columns.astype(int)
user_movie_matrix.to_csv('user_movie_matrix.csv', index=True)

In [1048]:
print(user_movie_matrix.head(3))

movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2          4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0  ...   
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movie_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                               
1          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3          NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[3 rows x 1682 columns]


Load the user-movie ratings matrix that was just created into `UM` using the `user_id` column as the index. Then create a boolean matrix with `True` for each movie that each user has not rated. This will be useful later for removing movies the user has already rated from the recommendations.

In [1049]:
UM = pd.read_csv('user_movie_matrix.csv', index_col='user_id')
UM.columns.name = 'movie_id'
UM.columns = UM.columns.astype(int)
unrated = UM.isna()

To use cosine similarity for our collaborative filtering, we need to give a rating to movies that have not been rated since they are current `NaN`. The two approaches I considered are filling `NaN` values with 0, or the mean value of the user. Both approaches seem to yield similar results for the recommendations, but based off of instinct I chose to use the mean value.

I think that users naturally gravitate towards movies that they are going to enjoy (based off recommendations, titles, trailers, sequels, etc.) which would raise their mean rating above their what their mean rating would be for a uniformly randomly chosen movie. In other words, I think that a user's mean rating over all the movies in the dataset would be lower than the mean rating over all the movies they have rated. However, I still think that the overall mean rating would be closer to the mean rating over the movies they have rated than to 0. Therefore, I think using the mean value is more useful in this context.

In [1050]:
# Replace NaN values.
# UM = UM.apply(lambda row: row.fillna(0), axis=1)
UM = UM.apply(lambda row: row.fillna(row.mean()), axis=1)

Each row is z-scale normalized by subtracting the mean of the row and scaling by the standard deviation. This centers each user's mean rating about 0 and normalizes the spread of their rating distribution. This is important because each user will have an average rating bias and use a different range of the full scale. In general, this makes user's ratings more comparable.

In [1051]:
# Apply z-scale normalization for each user's ratings.
UM_norm = UM.apply(lambda row: (row - row.mean())  / row.std(), axis=1)

In [1052]:
print(UM_norm.head(3).round(2))

movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1         2.74  -1.2  0.77  -1.2  -1.2  2.74  0.77 -5.14  2.74 -1.20  ...   
2         1.48  -0.0 -0.00  -0.0  -0.0 -0.00 -0.00 -0.00 -0.00 -8.71  ...   
3        -0.00  -0.0 -0.00  -0.0  -0.0 -0.00 -0.00 -0.00 -0.00 -0.00  ...   

movie_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                               
1          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2         -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  
3         -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  

[3 rows x 1682 columns]


The `UM` matrix will be used for finding similar users while the `MU` (the transpose of the `UM` matrix) will be used for finding movie-movie similarities.

In [1053]:
MU = UM.T

One thing that is unclear to me, but seems important, is how the z-scale normalization should be applied for the MU matrix. Should we normalize the movie scores, or the user scores, or both? Z-scale normalization for users removes user bias and scales for the spread of a user's ratings. Z-scale normalization across movie ratings removes how well a movie is recieved in general (i.e. the mean), and scales according to the std which I think would correspond to how polarizing a movie is. The third option is to do one and then the other or vise versa.

I have not come to a conclusion on this. However, I don't think that similarity between movies should be affected by how well the movie was recieved in general. Therefore, I will normalize across movie ratings, not across user ratings.

In [1054]:
MU_norm = MU.apply(lambda row: (row - row.mean())  / row.std(), axis=1)

Using the processed data, calculate the user-user and movie-movie similaries using cosine similarity.

In [1055]:
user_sim = pd.DataFrame(cosine_similarity(UM_norm), index=UM_norm.index, columns=UM_norm.index)
movie_sim = pd.DataFrame(cosine_similarity(MU_norm), index=MU_norm.index, columns=MU_norm.index)

Load the movie data (title, movie_id, release date, etc.) into the matrix `movies`. This will be necessary later for finding movie titles from movie IDs.

In [1056]:
movies = pd.read_csv('uitem.csv', index_col='movie_id')

## Part 2: Collaborative Filtering

### User-user

The following is the implementation of user-user collaborative filtering using the above processed data. The `recommend_movies_for_user` function does the following:
- Line 2: Find the similarities with all other users for `user_id`
- Line 2: Drop `user_id` since it will always be 1
- Line 2: Sort in descending order and take the first 50 most similar users and store in `sims`
- Line 3: Retrieve the movie ratings from all users in `sims` and store in `sim_ratings`
- Line 5: Get the predicted ratings of movies for `user_id` by finding the weighted sum of ratings of similar users in `sims` and store in `pred_ratings`
- Line 6: Remove all previously rated movies from `pred_ratings` using `unrated` so that only movies the user hasn't seen will be recommended
- Line 7: Sort `pred_ratings` in descending order and get the first `num` indexes of the recommended movies and store in `recommend_idx`
- Line 11: Return the recommended movie titles using `recommend_idx` to index the `movies` dataframe

In [1057]:
def recommend_movies_for_user(user_id, num=5):
    sims = user_sim[user_id].drop(user_id).sort_values(ascending=False).head(50)
    sim_ratings = UM_norm.loc[sims.index]
    
    pred_ratings = sim_ratings.T.dot(sims.T) / sims.sum()
    pred_ratings = pred_ratings.loc[unrated.loc[user_id]]
    recommend_idx = pred_ratings.sort_values(ascending=False).head(num).index

    movie_titles = movies.loc[recommend_idx, 'title'].to_list()

    result_df = pd.DataFrame({
    'Ranking': range(1, num+1),
    'Movie Name': movie_titles
    })
    result_df.set_index('Ranking', inplace=True)
    
    return result_df

print(recommend_movies_for_user(1, 10))

                                                Movie Name
Ranking                                                   
1                                  Schindler's List (1993)
2                                        Casablanca (1942)
3                                             Glory (1989)
4                                    Close Shave, A (1995)
5                                Lawrence of Arabia (1962)
6        Dr. Strangelove or: How I Learned to Stop Worr...
7                   One Flew Over the Cuckoo's Nest (1975)
8                Butch Cassidy and the Sundance Kid (1969)
9                                       Rear Window (1954)
10                               North by Northwest (1959)


### Item-item

The following is the implementation of movie–movie collaborative filtering using the processed movie similarity matrix. The `recommend_movies` function performs the following steps:

- Line 2: Look up the `movie_id` corresponding to the given `movie_name` by finding its index in `movies`
- Line 3: Retrieve similarity scores between the input movie and all other movies from the `movie_sim` matrix and drop the input movie since we don't want to recommend the same movie
- Line 4: Sort the similarity scores in descending order, select the top `num` most similar movies and store their indices as `sim_idx`
- Line 5: Use `sim_idx` to retrieve the corresponding movie titles from `movies` and return them as recommendations

In [1058]:
def recommend_movies(movie_name, num=5):
    movie_id = int(movies[movies["title"] == movie_name].index[0])
    sims = movie_sim.loc[movie_id].drop(movie_id)
    sim_idx = sims.sort_values(ascending=False).head(num).index.astype(int)

    movie_titles = movies.loc[sim_idx, 'title']
    
    result_df = pd.DataFrame({
    'Ranking': range(1, num+1),
    'Movie Name': movie_titles
    })
    result_df.set_index('Ranking', inplace=True)
    
    return result_df

print(recommend_movies("Star Wars (1977)", 10))

                                        Movie Name
Ranking                                           
1                        Return of the Jedi (1983)
2                  Empire Strikes Back, The (1980)
3                   Raiders of the Lost Ark (1981)
4        Indiana Jones and the Last Crusade (1989)
5                            Butterfly Kiss (1995)
6                           Man of the Year (1995)
7                              New York Cop (1996)
8                              Safe Passage (1994)
9                Destiny Turns on the Radio (1995)
10                    Foreign Correspondent (1940)


## Part 3: Pixie

### Data Processing
The following block processes and prepares movie rating data for the graph:

- Line 1: Loads the movie metadata from `uitem.csv`, sets `movie_id` as the index then resets the index so it becomes a regular column
- Line 2: Loads the user–movie rating matrix from `user_movie_matrix.csv` using `user_id` as the index
- Line 2: I select only the first 250 users just to speed up the process. More can be added, if desired, for more accuracy.
- Line 4: Converts the wide-format matrix into a long-format DataFrame by stacking ratings into individual rows
- Line 5: Renames the resulting columns to `user_id`, `movie_id`, and `rating`
- Line 6: Converts all `movie_id` values to integers for merging
- Line 7: Drops any rows where the rating is missing (i.e. `NaN`)
- Line 8: Merges the `ratings` with the `movies` to include movie titles based on `movie_id`


In [1059]:
movies = pd.read_csv('uitem.csv', index_col='movie_id').reset_index()
UM = pd.read_csv('user_movie_matrix.csv', index_col='user_id')
UM = UM.apply(lambda row: (row - row.mean())  / row.std(), axis=1)

ratings = UM.stack().reset_index()
ratings.columns = ['user_id', 'movie_id', 'rating']
ratings['movie_id'] = ratings['movie_id'].astype(int)
ratings = ratings.dropna()
ratings = ratings.merge(movies[['movie_id', 'title']], on='movie_id')
print(ratings)

       user_id  movie_id    rating                                 title
0            1         1  1.099812                      Toy Story (1995)
1            1         2 -0.482986                      GoldenEye (1995)
2            1         3  0.308413                     Four Rooms (1995)
3            1         4 -0.482986                     Get Shorty (1995)
4            1         5 -0.482986                        Copycat (1995)
...        ...       ...       ...                                   ...
99995      943      1067 -1.120605                  Bottle Rocket (1996)
99996      943      1074  0.468101                  Reality Bites (1994)
99997      943      1188 -0.326252                  Young Guns II (1990)
99998      943      1228 -0.326252  Under Siege 2: Dark Territory (1995)
99999      943      1330 -0.326252        An Unforgettable Summer (1994)

[100000 rows x 4 columns]


### Graph Building
The following block builds the bipartite graph used for the random walk. It simply iterates through all the edges (users and movies) in `ratings` and builds the graph in a dictionary, `graph`.

In [1060]:
graph = defaultdict(set)
for _, row in ratings.iterrows():
    user, movie = row['user_id'], row['title']
    graph[user].add(movie)
    graph[movie].add(user)

### Random Walk
The `random_walk` function simulates a random walk over a user–movie bipartite graph starting from a given node:
- Line 1: Initializes the walk from the `start_node` and sets up a dictionary to count movie visits.
- Line 3: Iterates for a fixed number of steps defined by `walk_length`.
- Line 4: Retrieves the list of neighbors connected to the current node.
- Line 5: If the current node has no neighbors, the walk terminates early.
- Line 7: Randomly selects the next node from the list of neighbors.
- Line 8: If the current node is a movie (i.e., a string), increments its visit count.
- Line 10: After the walk completes, returns a dictionary mapping movie titles to the number of times they were visited.

In [1166]:
def random_walk(graph, start_node, walk_length=100):
    current = start_node
    visit_counts = defaultdict(int)

    for _ in range(walk_length):
        neighbors = list(graph[current])
        if not neighbors:
            break

        current = random.choice(neighbors)
        if isinstance(current, str):
            visit_counts[current] += 1

    return visit_counts

### User Recommendation
The `recommend_for_user` function performs a random-walk-based recommendation for a given user:
- Line 1: Defines the function, specifying the graph, target `user_id`, total walk length, and number of recommendations to return
- Line 2: If the `user_id` is not present in the graph, returns an empty list
- Line 4: Performs a random walk starting from the given user node and records how frequently each movie is visited
- Line 5: Sorts the visited movies in descending order based on their visit count
- Line 6: Returns the top `top_n` most visited movie titles as recommendations

In [1074]:
def recommend_for_user(graph, user_id, walk_length=100, top_n=5):
    if user_id not in graph:
        return []

    visits = random_walk(graph, user_id, walk_length)
    
    rated_movies = graph[user_id]
    sorted_movies = sorted(
        [(movie, count) for movie, count in visits.items() if movie not in rated_movies],
        key=lambda x: x[1], reverse=True
    )

    top_movies = [movie for movie, _ in sorted_movies[:top_n]]
    result_df = pd.DataFrame({
        'Ranking': range(1, len(top_movies) + 1),
        'Movie Name': top_movies
    })
    result_df.set_index('Ranking', inplace=True)

    return result_df

In [1075]:
print(recommend_for_user(graph, 1, 100000, 10))

                           Movie Name
Ranking                              
1                    Liar Liar (1997)
2                       Scream (1996)
3         English Patient, The (1996)
4                Air Force One (1997)
5                      Titanic (1997)
6          Mission: Impossible (1996)
7             Schindler's List (1993)
8                       Ransom (1996)
9        Sense and Sensibility (1995)
10                  Saint, The (1997)


### Similar Movies
The `recommend_similar_movies` function performs a random-walk-based search to find movies similar to a given movie:
- Line 1: Defines the function with the graph, target `movie_title`, walk length, and number of similar movies to return
- Line 2: If the `movie_title` is not found in the graph, returns an empty list
- Line 4: Executes a random walk starting from the given movie node and records how frequently each movie is visited
- Line 5: Sorts the visited movies by visit count in descending order
- Line 6: Returns the top `top_n` most visited movies, excluding the input movie itself

In [1076]:
def recommend_similar_movies(graph, movie_title, walk_length=100, top_n=5):
    if movie_title not in graph:
        return []
    
    visits = random_walk(graph, movie_title, walk_length)
    sorted_movies = sorted(visits.items(), key=lambda x: x[1], reverse=True)
    top_movies = [movie for movie, _ in sorted_movies if movie != movie_title][:top_n]
    
    result_df = pd.DataFrame({
        'Ranking': range(1, len(top_movies) + 1),
        'Movie Name': top_movies
    })
    result_df.set_index('Ranking', inplace=True)
    
    return result_df

In [1077]:
print(recommend_similar_movies(graph, "Star Wars (1977)", 100000, 10))

                               Movie Name
Ranking                                  
1               Return of the Jedi (1983)
2                          Contact (1997)
3                        Liar Liar (1997)
4             English Patient, The (1996)
5                            Fargo (1996)
6                    Air Force One (1997)
7                     Pulp Fiction (1994)
8          Raiders of the Lost Ark (1981)
9        Silence of the Lambs, The (1991)
10                          Scream (1996)


### Weighted Graph Building
The following block builds the bipartite graph used for the **biased** random walk. It simply iterates through all the edges and for each `user_id` and `movie_title`, it stores the associated `movie_title` and `user_id` respectively with their associated rating to be used in the biased random walk.

The difference between the previous graph and this one is that this stores the associated weight for each edge. We can then bias the random walk towards movies or users that have higher ratings. Another change I made, that might be more efficient, is that I store the total weight for each user and movie in the graph. This prevents having to sum all of the weights every time we go to a new user or movie.

In [1140]:
movies = pd.read_csv('uitem.csv', index_col='movie_id').reset_index()
UM = pd.read_csv('user_movie_matrix.csv', index_col='user_id')
UM = UM.apply(lambda col: (col - col.mean()) / col.std(), axis=0)
UM = UM.apply(lambda col: (col - col.min()) / (col.max() - col.min()), axis=0)

ratings = UM.stack().reset_index()
ratings.columns = ['user_id', 'movie_id', 'rating']
ratings['movie_id'] = ratings['movie_id'].astype(int)
ratings = ratings.dropna()
ratings = ratings.merge(movies[['movie_id', 'title']], on='movie_id')

In [1141]:
graph_weight = defaultdict(dict)

for _, row in ratings.iterrows():
    user = row['user_id']
    movie = row['title']
    rating = row['rating']

    graph_weight[user][movie] = rating
    graph_weight[movie][user] = rating

### Biased Coin Random Walk

This function implements a biased coin random walk. At each step:
- A neighbor is randomly chosen.
- The walk accepts this neighbor based on a probability equal to the rating (between 0 and 1).
- If accepted the walk moves to that neighbor, otherwise, it retries with a new neighbor.
- If the current node is a movie title (i.e. a string), its visit count is incremented.

In [1142]:
def biased_coin_random_walk(graph, start_node, walk_length=100):
    current = start_node
    visit_counts = defaultdict(int)

    for i in range(walk_length):
        neighbors = list(graph.get(current, {}).keys())
        if not neighbors:
            break

        accepted = False
        while not accepted:
            neighbor = random.choice(neighbors)
            rating = graph[current][neighbor]
            acceptance_prob = rating

            if random.random() <= acceptance_prob:
                accepted = True
                current = neighbor

        if isinstance(current, str):
            visit_counts[current] += 1

    return visit_counts

This function recommends movies to the user using the biased coin random walk function
- It starts the walk from the given user and runs for a specified number of steps  
- At each step it randomly selects a neighbor and uses the rating as the probability of accepting the move  
- Movies visited during the walk are counted and stored  
- After the walk the movies are sorted by visit count and the top results are returned  
- The output is a dataframe showing the recommended movie titles and their ranking  

In [1147]:
def recommend_for_user_biased_coin(graph, user_id, walk_length=100, top_n=5):
    if user_id not in graph:
        return []

    visits = biased_coin_random_walk(graph, user_id, walk_length)
    sorted_movies = sorted(visits.items(), key=lambda x: x[1], reverse=True)
    top_movies = [movie for movie, _ in sorted_movies[:top_n]]

    result_df = pd.DataFrame({
        'Ranking': range(1, len(top_movies) + 1),
        'Movie Name': top_movies
    })
    result_df.set_index('Ranking', inplace=True)

    return result_df

In [1173]:
print(recommend_for_user_biased_coin(graph_weight, 1, 10000, 10))

                               Movie Name
Ranking                                  
1                        Star Wars (1977)
2                          Contact (1997)
3                     Forrest Gump (1994)
4                          Titanic (1997)
5               Return of the Jedi (1983)
6                     Pulp Fiction (1994)
7                        Toy Story (1995)
8        Silence of the Lambs, The (1991)
9                       Braveheart (1995)
10                   Air Force One (1997)


This function finds movies similar to a given movie using the biased coin random walk
- The walk starts from the input movie and continues for a fixed number of steps  
- At each step it randomly selects a connected user or movie and uses the rating as the acceptance probability  
- Movies visited during the walk are counted and tracked  
- The input movie is excluded from the results  
- After the walk movies are sorted by how often they were visited and the top matches are selected  
- The output is a dataframe listing the most similar movie titles and their ranking  

In [1145]:
def recommend_similar_movies_biased_coin(graph_weight, movie_title, walk_length=100, top_n=5):
    if movie_title not in graph_weight:
        return []

    visits = biased_coin_random_walk(graph_weight, movie_title, walk_length)
    sorted_movies = sorted(visits.items(), key=lambda x: x[1], reverse=True)
    top_movies = [movie for movie, _ in sorted_movies if movie != movie_title][:top_n]

    result_df = pd.DataFrame({
        'ranking': range(1, len(top_movies) + 1),
        'movie_name': top_movies
    })
    result_df.set_index('ranking', inplace=True)

    return result_df

In [1146]:
print(recommend_similar_movies_biased_coin(graph_weight, "Star Wars (1977)", 50000, 10))

                               movie_name
ranking                                  
1                            Fargo (1996)
2                          Contact (1997)
3               Return of the Jedi (1983)
4          Raiders of the Lost Ark (1981)
5                        Toy Story (1995)
6                    Jerry Maguire (1996)
7              Princess Bride, The (1987)
8             English Patient, The (1996)
9        Silence of the Lambs, The (1991)
10        Empire Strikes Back, The (1980)
